In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Cargar los datos de generación y meteorológicos
plant_1_generation = pd.read_csv('Plant_1_Generation_Data.csv')
plant_1_weather = pd.read_csv('Plant_1_Weather_Sensor_Data.csv')

# Copiar los datos de generación para mantener df_GD1 limpio
df_GD1 = plant_1_generation.copy()

# Aplicar el mapeo a los nombres de los paneles solares
unique_source_keys_list = df_GD1['SOURCE_KEY'].unique()
source_key_mapping = {key: f"Solar_Panel_{i+1}" for i, key in enumerate(unique_source_keys_list)}
df_GD1['SOURCE_KEY'] = df_GD1['SOURCE_KEY'].map(source_key_mapping)

# Convertir la columna 'DATE_TIME' a formato de fecha y hora
df_GD1['DATE_TIME'] = pd.to_datetime(df_GD1['DATE_TIME'], format='%d-%m-%Y %H:%M')
plant_1_weather['DATE_TIME'] = pd.to_datetime(plant_1_weather['DATE_TIME'], format='%Y-%m-%d %H:%M:%S')

# Unir los datos meteorológicos al dataframe de generación en función de la fecha y hora
df_GD1_with_weather = pd.merge(df_GD1, plant_1_weather, on='DATE_TIME', how='left')

# Eliminar las columnas PLANT_ID_y y SOURCE_KEY_y y renombrar las columnas PLANT_ID_x y SOURCE_KEY_x
df_GD1_with_weather_clean = df_GD1_with_weather.drop(columns=['PLANT_ID_y', 'SOURCE_KEY_y','PLANT_ID_x'])

# Renombrar las columnas para eliminar el sufijo '_x'
df_GD1_with_weather_clean = df_GD1_with_weather_clean.rename(columns={'SOURCE_KEY_x': 'SOURCE_KEY'})

nulos = df_GD1_with_weather_clean.isnull().sum()

# Mostrar cuántos valores nulos hay por columna
print(nulos)

# Rellenar los valores nulos en solo las columnas numéricas con la media
numerical_cols = df_GD1_with_weather_clean.select_dtypes(include=['float64', 'int64']).columns #Estp para quitar los floats
df_GD1_with_weather_clean[numerical_cols] = df_GD1_with_weather_clean[numerical_cols].fillna(df_GD1_with_weather_clean[numerical_cols].mean())

df_GD1_limpio = df_GD1_with_weather_clean

DATE_TIME              0
SOURCE_KEY             0
DC_POWER               0
AC_POWER               0
DAILY_YIELD            0
TOTAL_YIELD            0
AMBIENT_TEMPERATURE    4
MODULE_TEMPERATURE     4
IRRADIATION            4
dtype: int64


In [3]:
# Cargar los datos de generación y meteorológicos para la planta 2
plant_2_generation = pd.read_csv('Plant_2_Generation_Data.csv')
plant_2_weather = pd.read_csv('Plant_2_Weather_Sensor_Data.csv')

# Copiar los datos de generación para mantener df_GD2 limpio
df_GD2 = plant_2_generation.copy()

# Aplicar el mapeo a los nombres de los paneles solares
unique_source_keys_list_2 = df_GD2['SOURCE_KEY'].unique()
source_key_mapping_2 = {key: f"Solar_Panel_{i+1}" for i, key in enumerate(unique_source_keys_list_2)}
df_GD2['SOURCE_KEY'] = df_GD2['SOURCE_KEY'].map(source_key_mapping_2)

# Convertir la columna 'DATE_TIME' a formato de fecha y hora
# Ajustar el formato de fecha según sea necesario para que coincida con el archivo de la planta 2
df_GD2['DATE_TIME'] = pd.to_datetime(df_GD2['DATE_TIME'], format='%Y-%m-%d %H:%M:%S')
plant_2_weather['DATE_TIME'] = pd.to_datetime(plant_2_weather['DATE_TIME'], format='%Y-%m-%d %H:%M:%S')

# Unir los datos meteorológicos al dataframe de generación en función de la fecha y hora
df_GD2_with_weather = pd.merge(df_GD2, plant_2_weather, on='DATE_TIME', how='left')

# Eliminar las columnas PLANT_ID_y y SOURCE_KEY_y y renombrar las columnas PLANT_ID_x y SOURCE_KEY_x
df_GD2_with_weather_clean = df_GD2_with_weather.drop(columns=['PLANT_ID_y', 'SOURCE_KEY_y', 'PLANT_ID_x'])

# Renombrar las columnas para eliminar el sufijo '_x'
df_GD2_with_weather_clean = df_GD2_with_weather_clean.rename(columns={'SOURCE_KEY_x': 'SOURCE_KEY'})

# Mostrar cuántos valores nulos hay por columna
nulos2 = df_GD2_with_weather_clean.isnull().sum()
print("Valores nulos en cada columna (planta 2):\n", nulos2)

# Rellenar los valores nulos en solo las columnas numéricas con la media
numerical_cols_2 = df_GD2_with_weather_clean.select_dtypes(include=['float64', 'int64']).columns
df_GD2_with_weather_clean[numerical_cols_2] = df_GD2_with_weather_clean[numerical_cols_2].fillna(df_GD2_with_weather_clean[numerical_cols_2].mean())

# Guardar el dataframe limpio de la planta 2
df_GD2_limpio = df_GD2_with_weather_clean

Valores nulos en cada columna (planta 2):
 DATE_TIME              0
SOURCE_KEY             0
DC_POWER               0
AC_POWER               0
DAILY_YIELD            0
TOTAL_YIELD            0
AMBIENT_TEMPERATURE    0
MODULE_TEMPERATURE     0
IRRADIATION            0
dtype: int64


In [5]:
from tensorflow.keras.models import load_model
import joblib

# Cargar los modelos y escaladores de cada planta
Modelo_SP1 = load_model('Modelo_SP1.h5')
scaler_ker1 = joblib.load('scaler_ker1.pkl')

Modelo_SP2 = load_model('Modelo_SP2.h5')
scaler_ker2 = joblib.load('scaler_ker2.pkl')

features = ['AC_POWER', 'DAILY_YIELD', 'TOTAL_YIELD','AMBIENT_TEMPERATURE', 
            'MODULE_TEMPERATURE', 'IRRADIATION']
# Definir X (variables predictoras) y Y (variable objetivo)
X_ker_1 = df_GD1_limpio[features].values
X_ker_2 = df_GD2_limpio[features].values

from sklearn.model_selection import train_test_split

# Planta 1: Divide en entrenamiento y prueba
X_train_plant1, X_test_plant1, Y_train_plant1, Y_test_plant1 = train_test_split(X_ker_1, df_GD1_limpio['DC_POWER'].values, 
                                                                                test_size=0.2, random_state=42)

# Planta 2: Divide en entrenamiento y prueba
X_train_plant2, X_test_plant2, Y_train_plant2, Y_test_plant2 = train_test_split(X_ker_2, df_GD2_limpio['DC_POWER'].values, 
                                                                                test_size=0.2, random_state=42)

# Función para realizar el ensamble
def predict_ensemble(X_ker_1, X_ker_2):
    min_samples = min(len(X_ker_1), len(X_ker_2))
    X_ker_1_sub = X_ker_1[:min_samples]
    X_ker_2_sub = X_ker_2[:min_samples]
    
    # Escalar y predecir con cada modelo
    X_plant1_scaled = scaler_ker1.transform(X_ker_1_sub)
    X_plant2_scaled = scaler_ker2.transform(X_ker_2_sub)
    
    pred_plant1 = Modelo_SP1.predict(X_plant1_scaled)
    pred_plant2 = Modelo_SP2.predict(X_plant2_scaled)
    
    # Ensamble (promedio de las predicciones)
    combined_prediction = (pred_plant1 + pred_plant2) / 2
    
    return combined_prediction


# Ejemplo de predicción
# X_test_plant1 y X_test_plant2 son las características de prueba de cada planta
Y_pred_ensemble = predict_ensemble(X_test_plant1, X_test_plant2)


424/424 ━━━━━━━━━━━━━━━━━━━━ 0s 241us/step
424/424 ━━━━━━━━━━━━━━━━━━━━ 0s 242us/step


In [7]:
min_samples = min(len(X_test_plant1), len(X_test_plant2))

# Reducir ambas matrices a un número común de muestras
X_test_plant1_sub = X_test_plant1[:min_samples]
X_test_plant2_sub = X_test_plant2[:min_samples]

# Escalar los subconjuntos
X_test_plant1_scaled = scaler_ker1.transform(X_test_plant1_sub)
X_test_plant2_scaled = scaler_ker2.transform(X_test_plant2_sub)

In [8]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Seleccionar el tamaño mínimo de muestras entre ambas plantas
min_samples = min(len(Y_test_plant1), len(Y_test_plant2))
Y_test_plant1_sub = Y_test_plant1[:min_samples]
Y_test_plant2_sub = Y_test_plant2[:min_samples]

# Promediar los valores reales de ambas plantas para comparar con el ensamble
Y_test_combined = (Y_test_plant1_sub + Y_test_plant2_sub) / 2

# Calcular métricas de rendimiento
mse = mean_squared_error(Y_test_combined, Y_pred_ensemble)
mae = mean_absolute_error(Y_test_combined, Y_pred_ensemble)
r2 = r2_score(Y_test_combined, Y_pred_ensemble)

print("Resultados del modelo ensamblado:")
print("MSE:", mse)
print("MAE:", mae)
print("R²:", r2)

Resultados del modelo ensamblado:
MSE: 1128.3663189779902
MAE: 19.04867500726633
R²: 0.9997231401010581


Resultados del modelo ensamblado:

MSE: 690.581889483876

MAE: 15.555942529120001

R²: 0.999830556416903